In [ ]:
# connecting my drive to this google colab nb

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# loading the capstone dataset folder from my drive and printing the files under it

import os

dataset_path = '/content/drive/MyDrive/Capstone Preprocessing/B2C'
os.listdir(dataset_path)  # List files in the dataset folder

['b2c_sales_v5',
 'bank_statements1',
 'b2c_sales_cleaned',
 'bank_statements1_preprocessed']

"""
Data Preprocessing Script for B2C Sales and Bank Statements.

This script reads raw CSV files from specified input directories:
- B2C Sales: Expected columns include 'Date', 'Type', 'Place Of Supply', 'Rate',
             'Applicable Rate (%)', 'Taxable Value', 'Cess Amount', 'E-Commerce GSTIN'.
             It processes rates like 'X+Y', fills missing GSTIN with 'null', cleans
             data types, handles missing numerics, adds date features, checks duplicates,
             and sorts.
- Bank Statements: Expected columns include 'Date', 'Narration', 'Chq./Ref.No.',
                  'Value Dt', 'Withdrawal Amt.', 'Deposit Amt.', 'Closing Balance'.
                  It handles opening balance rows, cleans dates and amounts, checks
                  reference uniqueness, calculates balance, adds features, checks
                  duplicates, and sorts.

It keeps the original column names from the input files and adds new
standardized feature columns. The preprocessed data is saved to specified
output directories. Designed for clarity and potential use in environments
like Google Colab.
"""

In [ ]:
# =============================================================================
# SCRIPT: Bank Statement Preprocessing
# PURPOSE: Reads raw bank statement CSV files, preprocesses them, selects
#          final columns, and saves the cleaned data to a separate
#          output directory.
# =============================================================================

# =============================================================================
# 1. IMPORTS
# =============================================================================
print("--- 1. Loading Libraries ---")
import os
import pandas as pd
import numpy as np
import re # Useful for cleaning values
from io import StringIO # For capturing df.info() output
import traceback # For detailed error printing
# from google.colab import drive # Uncomment if using Google Colab
# drive.mount('/content/drive') # Uncomment if using Google Colab

# =============================================================================
# 2. CONFIGURATION (BANK SPECIFIC)
# =============================================================================
print("--- 2. Setting Bank Statement Configuration ---")

# --- Input/Output Directories ---
# Adjust these paths if not using Google Drive or if your structure differs
BASE_DIR = '/content/drive/MyDrive/Capstone Preprocessing/B2C' # Example base if using Colab
bank_input_dir = os.path.join(BASE_DIR, 'bank_statements1')
bank_output_dir = os.path.join(BASE_DIR, 'bank_preprocessed1')

# --- Create Bank Output Directory ---
print(f"     Creating Bank output directory (if needed): {os.path.abspath(bank_output_dir)}")
os.makedirs(bank_output_dir, exist_ok=True)

# --- Bank Statement Column Names ---
# !!! CHECK THESE REQUIRED BANK NAMES AGAINST YOUR ACTUAL FILE HEADER !!!
REQUIRED_BANK_COLS = ['Date', 'Narration', 'Chq./Ref.No.', 'Value Dt',
                       'Withdrawal Amt.', 'Deposit Amt.', 'Closing Balance']
# --- Desired FINAL Bank Columns ---
FINAL_BANK_COLS = ['Date', 'Narration', 'Chq./Ref.No.', 'Value Dt',
                   'Withdrawal Amt.', 'Deposit Amt.', 'Closing Balance']


print(f"     Bank Input Directory: {os.path.abspath(bank_input_dir)}")
print(f"     Bank Output Directory: {os.path.abspath(bank_output_dir)}")
print(f"     Required Bank Columns: {REQUIRED_BANK_COLS}") # Check these!
print(f"     Desired Final Bank Columns: {FINAL_BANK_COLS}") # Check these!


# =============================================================================
# 3. BANK STATEMENT PREPROCESSING FUNCTION
# =============================================================================
print("\n--- 3. Defining Bank Statement Preprocessing Function ---")

def preprocess_bank_statement_final_cols(df, filename=""):
    """
    Preprocesses a bank statement DataFrame and keeps only specified final columns.

    Performs the following steps:
    1. Verifies required columns (as defined in REQUIRED_BANK_COLS) exist. *** Check REQUIRED_BANK_COLS ***
    2. Separates a potential 'Opening Balance' row.
    3. Cleans and converts date columns ('Date', 'Value Dt') to datetime (DD-MM-YYYY format). Drops rows with invalid primary 'Date'.
    4. Cleans and converts amount columns ('Withdrawal Amt.', 'Deposit Amt.', 'Closing Balance') to numeric.
    5. Cleans text columns ('Narration', 'Chq./Ref.No.'), strips whitespace, handles NaNs.
    6. Checks for uniqueness of 'Chq./Ref.No.'.
    7. Handles missing values (fills 'Value Dt', amounts with 0, text with placeholders).
    8. Checks for and removes duplicate transaction rows.
    9. Adds temporary features: 'year', 'month', 'day', 'day_of_week', 'transaction_type'.
    10. Attempts to recalculate 'Closing Balance' into a temporary column.
    11. Reattaches the Opening Balance row.
    12. Sorts the DataFrame by 'Date'.
    13. Selects only the columns specified in FINAL_BANK_COLS.
    14. Performs a final inspection.

    Args:
        df (pd.DataFrame): The raw bank statement DataFrame.
        filename (str, optional): Original filename for logging purposes. Defaults to "".

    Returns:
        pd.DataFrame or None: The preprocessed DataFrame containing only FINAL_BANK_COLS,
                              or None if critical errors occur.
    """
    print(f"\n--- Starting Bank Statement Preprocessing for: {filename} ---")
    # --- Initial Checks ---
    if df.empty:
        print("  WARN: Input DataFrame is empty. Skipping.")
        return None

    df_processed = df.copy()
    print(f"     Initial raw columns: {df_processed.columns.tolist()}")

    # --- Step 1: Verify Required Columns ---
    print("  1. Verifying required columns...")
    actual_cols = df_processed.columns.tolist()
    # Trim whitespace from actual column names for comparison
    actual_cols_trimmed = {col.strip(): col for col in actual_cols}
    df_processed.columns = [col.strip() for col in actual_cols] # Apply trimming
    actual_cols = df_processed.columns.tolist() # Update list

    missing_req = [col for col in REQUIRED_BANK_COLS if col not in actual_cols_trimmed]
    if missing_req:
        print(f"  ERROR: Missing REQUIRED Bank columns: {missing_req}. Required are {REQUIRED_BANK_COLS}. Found columns: {actual_cols}. Cannot proceed. Skipping file.")
        return None
    print(f"     Found required Bank columns: {REQUIRED_BANK_COLS}")

    # Use potentially trimmed names
    date_col = 'Date'
    narr_col = 'Narration'
    ref_col = 'Chq./Ref.No.'
    val_dt_col = 'Value Dt'
    wd_amt_col = 'Withdrawal Amt.'
    dp_amt_col = 'Deposit Amt.'
    bal_col = 'Closing Balance'

    # --- Step 2: Handle Opening Balance Row ---
    print("  2. Checking for and separating Opening Balance row...")
    opening_balance_row = None
    if narr_col in df_processed.columns and not df_processed.empty:
        # Ensure the first narration value is treated as a string before processing
        first_narration_val = df_processed[narr_col].iloc[0]
        first_narration = str(first_narration_val).strip().lower() if pd.notna(first_narration_val) else ""

        # More robust check for opening balance narration
        ob_keywords = ['opening balance', 'o/b', 'opng bal', 'op balance', 'brought forward', 'bal b/f']
        if any(keyword in first_narration for keyword in ob_keywords):
             # Check if Date is null or if BOTH Withdrawal and Deposit are null/zero
             first_date_is_null = pd.isna(df_processed[date_col].iloc[0]) if date_col in df_processed.columns else True

             # Check amounts carefully - handle cases where columns might not exist yet
             wd_val = 0
             dp_val = 0
             if wd_amt_col in df_processed.columns and pd.notna(df_processed[wd_amt_col].iloc[0]):
                 wd_val = pd.to_numeric(str(df_processed[wd_amt_col].iloc[0]).replace(',','').strip(), errors='coerce')
                 wd_val = wd_val if pd.notna(wd_val) else 0 # Treat parsing failure as 0 for this check
             if dp_amt_col in df_processed.columns and pd.notna(df_processed[dp_amt_col].iloc[0]):
                 dp_val = pd.to_numeric(str(df_processed[dp_amt_col].iloc[0]).replace(',','').strip(), errors='coerce')
                 dp_val = dp_val if pd.notna(dp_val) else 0 # Treat parsing failure as 0 for this check

             first_wd_dp_are_zero_or_null = (wd_val == 0 and dp_val == 0)

             # Separate if date is null OR wd/dp are zero/null AND narration matches keywords
             if first_date_is_null or first_wd_dp_are_zero_or_null:
                 opening_balance_row = df_processed.iloc[0:1].copy()
                 df_processed = df_processed.iloc[1:].copy().reset_index(drop=True)
                 print(f"     Detected and separated potential Opening Balance row based on Narration ('{first_narration}') and missing Date or zero/missing Amounts.")
             else:
                 print(f"     First row narration ('{first_narration}') contains keywords, but Date is present and Amounts are non-zero. Not separating automatically.")
        else:
            print(f"     No distinct Opening Balance pattern detected in first row Narration ('{first_narration}').")

    # --- Step 3: Handle Data Types and Cleaning ---
    print("  3. Processing Data Types & Cleaning...")
    # Process Date Columns
    date_cols_to_process = [date_col, val_dt_col]
    for col in date_cols_to_process:
        if col in df_processed.columns:
            print(f"     Processing '{col}' column...")
            original_nan_count = df_processed[col].isnull().sum()
            # Convert to string first
            df_processed[col] = df_processed[col].astype(str).str.strip()
            df_processed[col].replace(['-', 'nil', 'null', 'N/A', 'NA', 'NaN', 'NaT', ''], np.nan, inplace=True)

            # Try specific format first (DD-MM-YYYY)
            dates_parsed = pd.to_datetime(df_processed[col], format='%d-%m-%Y', errors='coerce')
            invalid_mask_fmt = dates_parsed.isna()
            print(f"       Invalid dates after DD-MM-YYYY format: {invalid_mask_fmt.sum()}")

            # Try generic parser for those that failed the specific format
            if invalid_mask_fmt.any():
                 print(f"       Trying generic date parser (dayfirst=True) for {invalid_mask_fmt.sum()} failed dates...")
                 # Apply generic only where specific format failed
                 dates_generic = pd.to_datetime(df_processed.loc[invalid_mask_fmt, col], errors='coerce', dayfirst=True)
                 dates_parsed.loc[invalid_mask_fmt] = dates_generic # Update main series
                 print(f"       Invalid dates remaining after generic pass: {dates_parsed.isna().sum()}")

            df_processed[col] = dates_parsed # Assign back

            converted_nan_count = df_processed[col].isnull().sum()
            # Note: NaNs could decrease if placeholder strings were converted to NaT then parsed correctly later
            print(f"       NaNs/NaTs before parsing: {original_nan_count}, After parsing: {converted_nan_count}")

    # Drop rows if primary 'Date' is invalid AFTER parsing attempts
    rows_before_drop = len(df_processed)
    if date_col in df_processed.columns:
        df_processed.dropna(subset=[date_col], inplace=True)
        rows_removed = rows_before_drop - len(df_processed)
        if rows_removed > 0:
             print(f"       Removed {rows_removed} rows with invalid/missing primary '{date_col}' after parsing attempts.")
    else:
         print(f"     WARN: Cannot drop rows based on missing dates as '{date_col}' column not found.")

    # Check if only OB row remains or no data
    if df_processed.empty and opening_balance_row is None:
        print(f"  ERROR: No valid transaction data remaining after cleaning '{date_col}'. Skipping file.")
        return None
    elif df_processed.empty and opening_balance_row is not None:
        print(f"  WARN: Only Opening Balance row remains after cleaning '{date_col}'. Proceeding with OB row only.")
        # Ensure required cols exist in OB row if it's the only thing left
        for req_col in REQUIRED_BANK_COLS:
             if req_col not in opening_balance_row.columns: opening_balance_row[req_col] = np.nan

    # Process Amount Columns
    amount_cols_to_process = [wd_amt_col, dp_amt_col, bal_col]
    for col in amount_cols_to_process:
        if col in df_processed.columns:
            print(f"     Processing '{col}' column (converting from string)...")
            original_nan_count = df_processed[col].isnull().sum()
            df_processed[col] = df_processed[col].astype(str).str.strip()
            # Handle blanks, hyphens, common text placeholders
            df_processed[col].replace(['', '-', 'nil', 'null', 'N/A', 'NA', 'NaN', 'NaT'], np.nan, inplace=True)
            # Remove commas used as thousand separators
            df_processed[col] = df_processed[col].str.replace(',', '', regex=False)
            # Specific check for credit notations like ' Cr' or ' Dr' (remove them) - adjust if needed
            df_processed[col] = df_processed[col].str.replace(r'\s*(Cr|Dr)\.?$', '', regex=True)

            df_processed[col] = pd.to_numeric(df_processed[col], errors='coerce')
            converted_nan_count = df_processed[col].isnull().sum()
            # Note: NaNs could decrease if placeholder strings were converted to NaN then parsed correctly
            print(f"       NaNs before numeric conversion: {original_nan_count}, After conversion: {converted_nan_count}")
        elif col in REQUIRED_BANK_COLS: # Check if a required amount col is missing entirely
            print(f"     WARN: Required amount column '{col}' not found in DataFrame. Will be missing from output.")


    # Process Text Columns
    text_cols_to_process = [narr_col, ref_col]
    for col in text_cols_to_process:
        if col in df_processed.columns:
            print(f"     Cleaning '{col}' column...")
            # Convert to string, strip whitespace
            df_processed[col] = df_processed[col].astype(str).str.strip()
            # Replace specific placeholder strings (case-insensitive check then replace)
            null_like_vals = ['nan', 'none', 'null', 'na', 'n/a', '#n/a', '<na>', 'nat', '-', '']
            mask = df_processed[col].str.lower().isin(null_like_vals) | df_processed[col].isnull()
            # Count how many will be replaced before doing it
            num_to_replace = mask.sum()
            if num_to_replace > 0:
                 print(f"       Replacing {num_to_replace} placeholder/null values in '{col}' with NaN.")
                 df_processed.loc[mask, col] = np.nan
            else:
                 print(f"       No common placeholder values found to replace in '{col}'.")
        elif col in REQUIRED_BANK_COLS:
             print(f"     WARN: Required text column '{col}' not found in DataFrame. Will be missing from output.")


    # --- Step 4: Check Uniqueness of Reference Numbers ---
    print(f"  4. Checking uniqueness of '{ref_col}'...")
    if ref_col in df_processed.columns:
        # Check uniqueness *after* cleaning, excluding NaNs resulting from cleaning
        refs_to_check = df_processed[ref_col].dropna().astype(str)

        if not refs_to_check.empty:
            ref_counts = refs_to_check.value_counts()
            non_unique_refs = ref_counts[ref_counts > 1]
            if not non_unique_refs.empty:
                print(f"    WARN: Found {len(non_unique_refs)} distinct reference numbers that are NOT unique (totaling {non_unique_refs.sum()} occurrences) in '{ref_col}' after cleaning.")
                # print(f"      Examples (top 5):\n{non_unique_refs.head(5).to_string()}") # Optional: print examples
            else:
                print(f"    INFO: All non-missing values in '{ref_col}' appear unique after cleaning.")
        else:
            print(f"    INFO: '{ref_col}' column contains only missing values after cleaning.")
    else:
        print(f"    INFO: Column '{ref_col}' not found, skipping uniqueness check.")


    # --- Step 5: Handle Missing Values (Post Cleaning) ---
    print("  5. Handling remaining missing values...")
    # Fill missing 'Value Dt' with 'Date'
    if val_dt_col in df_processed.columns and date_col in df_processed.columns:
        missing_value_dt = df_processed[val_dt_col].isnull().sum()
        if missing_value_dt > 0:
            if pd.api.types.is_datetime64_any_dtype(df_processed[date_col]):
                 print(f"     Filling {missing_value_dt} missing '{val_dt_col}' with corresponding '{date_col}'.")
                 df_processed[val_dt_col].fillna(df_processed[date_col], inplace=True)
            else:
                 print(f"     WARN: Cannot fill missing '{val_dt_col}' because '{date_col}' is not a valid datetime column after cleaning. NaNs remain.")

    # Fill missing Amounts (Withdrawal/Deposit) with 0 (after numeric conversion)
    for col in [wd_amt_col, dp_amt_col]:
        if col in df_processed.columns:
             missing_amount = df_processed[col].isnull().sum()
             if missing_amount > 0:
                 print(f"     Filling {missing_amount} missing '{col}' values with 0.")
                 df_processed[col].fillna(0, inplace=True)

    # Warn about missing 'Closing Balance' - DON'T fill with 0 unless calculated later
    if bal_col in df_processed.columns:
        missing_balance = df_processed[bal_col].isnull().sum()
        if missing_balance > 0:
             # Changed from WARN to INFO as recalculation might fix it
             print(f"     INFO: {missing_balance} rows have missing '{bal_col}' after conversion. Will remain NaN unless calculation succeeds.")

    # Fill missing text fields (Narration/RefNo) with placeholders *after* cleaning
    for col in [narr_col, ref_col]:
         if col in df_processed.columns:
             missing_text = df_processed[col].isnull().sum() # Check NaNs again
             if missing_text > 0:
                 fill_value = 'Unknown Narration' if col == narr_col else 'No Ref'
                 print(f"     Filling {missing_text} remaining missing '{col}' with '{fill_value}'.")
                 df_processed[col].fillna(fill_value, inplace=True)


    # --- Step 6: Handle Duplicate Transaction Rows ---
    print("  6. Checking for duplicate transaction rows...")
    # Use required cols for duplicate check as they should always exist and be filled
    key_cols_for_dup_check = [date_col, narr_col, wd_amt_col, dp_amt_col, ref_col]
    existing_key_cols = [c for c in key_cols_for_dup_check if c in df_processed.columns]

    if len(existing_key_cols) >= 4 and not df_processed.empty: # Require at least Date + Narration + Amounts for meaningful check
        duplicates_count = df_processed.duplicated(subset=existing_key_cols).sum()
        if duplicates_count > 0:
            print(f"     Found and removing {duplicates_count} duplicate transaction rows based on {existing_key_cols}.")
            df_processed.drop_duplicates(subset=existing_key_cols, keep='first', inplace=True)
        else:
            print(f"     No duplicate transaction rows found based on key columns: {existing_key_cols}.")
    elif df_processed.empty:
        print("     Skipping duplicate check as DataFrame is empty (excluding potential OB row).")
    else:
        print(f"     Skipping duplicate check as not enough key columns ({existing_key_cols}) are available.")


    # --- Step 7: Feature Engineering (Temporary) ---
    print("  7. Adding temporary features (date components, transaction type)...")
    calculated_balance_col = 'calculated_balance' # Define name for potential new column
    # Initialize columns first
    df_processed['year'], df_processed['month'], df_processed['day'], df_processed['day_of_week'] = [pd.NA] * 4
    df_processed['transaction_type'] = 'Unknown' # Initialize

    if not df_processed.empty: # Only proceed if there are transaction rows
        # Add Date Features
        if date_col in df_processed.columns and pd.api.types.is_datetime64_any_dtype(df_processed[date_col]):
            df_processed['year'] = df_processed[date_col].dt.year
            df_processed['month'] = df_processed[date_col].dt.month
            df_processed['day'] = df_processed[date_col].dt.day
            df_processed['day_of_week'] = df_processed[date_col].dt.dayofweek # Monday=0, Sunday=6
            print("     Added temporary date features.")
        else:
            print(f"    WARN: Cannot extract date features because '{date_col}' is not a valid datetime column.")

        # Add Transaction Type
        # Use already cleaned/filled numeric amounts (filled with 0 in Step 5)
        wd_present = wd_amt_col in df_processed.columns
        dp_present = dp_amt_col in df_processed.columns

        # Ensure columns exist before accessing
        wd_num = df_processed[wd_amt_col] if wd_present else pd.Series(0, index=df_processed.index)
        dp_num = df_processed[dp_amt_col] if dp_present else pd.Series(0, index=df_processed.index)

        # Determine transaction type based on non-zero amounts
        df_processed.loc[wd_num > 0, 'transaction_type'] = 'Debit'
        df_processed.loc[dp_num > 0, 'transaction_type'] = 'Credit'
        # Handle cases where both might be zero (e.g., balance adjustments, informational lines)
        df_processed.loc[(wd_num == 0) & (dp_num == 0), 'transaction_type'] = 'Zero Amount'
        print("     Added temporary transaction type.")
        # Optional: Print distribution for verification
        # print(f"      Transaction Type distribution:\n{df_processed['transaction_type'].value_counts().to_string()}")
    else:
        print("     Skipping temporary feature engineering as DataFrame (excluding OB row) is empty.")


    # --- Step 8: Recalculate Closing Balance (Optional, creates temporary column) ---
    print("  8. Attempting to recalculate closing balance (into temporary column)...")
    calculated_balance_added = False
    opening_balance_numeric = np.nan

    # Prerequisites: OB row exists, balance col in OB, WD/DP cols in main df, main df not empty
    if opening_balance_row is not None and \
       bal_col in opening_balance_row.columns and \
       wd_amt_col in df_processed.columns and \
       dp_amt_col in df_processed.columns and \
       not df_processed.empty:

        try:
            # Try to parse OB from the separated row (apply same cleaning as main df)
            ob_val_str = str(opening_balance_row[bal_col].iloc[0]).strip().replace(',', '')
            ob_val_str = re.sub(r'\s*(Cr|Dr)\.?$', '', ob_val_str) # Remove Cr/Dr
            opening_balance_numeric = pd.to_numeric(ob_val_str, errors='coerce')

            if pd.notna(opening_balance_numeric):
                 print(f"     Using parsed Opening Balance: {opening_balance_numeric:.2f}")

                 # Ensure sorting before calculation for correctness on same-day transactions
                 sort_cols = [date_col]
                 # Optional: Sort by Value Dt as secondary if reliable
                 # if val_dt_col in df_processed.columns and pd.api.types.is_datetime64_any_dtype(df_processed[val_dt_col]):
                 #    sort_cols.append(val_dt_col)

                 # Add temporary index for stable sort if original index isn't unique or reliable, then remove
                 temp_index_needed = not df_processed.index.is_unique
                 if temp_index_needed:
                     df_processed['__temp_sort_idx__'] = range(len(df_processed))
                     sort_cols.append('__temp_sort_idx__')

                 if date_col in df_processed.columns and pd.api.types.is_datetime64_any_dtype(df_processed[date_col]):
                     print(f"       Sorting by {sort_cols} for calculation...")
                     # Use stable sort (mergesort)
                     df_processed = df_processed.sort_values(by=sort_cols, kind='mergesort', na_position='last')
                 else:
                     print("     WARN: Cannot sort reliably for balance calculation as primary date column is invalid.")

                 # Remove temporary index if added
                 if temp_index_needed:
                     df_processed.drop(columns=['__temp_sort_idx__'], inplace=True)

                 # Use already cleaned numeric columns (filled with 0 in Step 5)
                 deposit_numeric = df_processed[dp_amt_col]
                 withdrawal_numeric = df_processed[wd_amt_col]

                 # Calculate running balance
                 df_processed[calculated_balance_col] = opening_balance_numeric + \
                    (deposit_numeric - withdrawal_numeric).cumsum()
                 calculated_balance_added = True
                 print(f"     Successfully added temporary '{calculated_balance_col}' column.")

                 # Compare with original balance if it exists and is numeric
                 if bal_col in df_processed.columns:
                      original_bal_numeric = df_processed[bal_col] # Already converted to numeric or NaN
                      # Create comparison dataframe excluding rows where either is NaN
                      comparison_df = pd.DataFrame({
                          'original': original_bal_numeric,
                          'calculated': df_processed[calculated_balance_col]
                      }).dropna()

                      if not comparison_df.empty:
                          diff = (comparison_df['calculated'] - comparison_df['original']).abs()
                          tolerance = 0.01 # Define tolerance for float comparison
                          mismatches = diff[diff > tolerance]
                          print(f"     Comparison with original '{bal_col}' ({len(comparison_df)} comparable rows): Max diff: {diff.max():.2f}, Mean diff: {diff.mean():.2f}")
                          if not mismatches.empty:
                              print(f"       WARN: Found {len(mismatches)} rows where calculated balance differs (> {tolerance:.2f}) from original.")
                          else:
                              print(f"       Calculated balance matches original balance (within {tolerance:.2f} tolerance) where comparable.")

                          # Optionally: Consider replacing original balance with calculated if original is missing
                          # missing_orig_mask = df_processed[bal_col].isnull() & df_processed[calculated_balance_col].notnull()
                          # if missing_orig_mask.any():
                          #     print(f"       INFO: Filling {missing_orig_mask.sum()} missing original balances with calculated values.")
                          #     df_processed.loc[missing_orig_mask, bal_col] = df_processed.loc[missing_orig_mask, calculated_balance_col]
                      else:
                          print(f"     Cannot compare balances (original '{bal_col}' missing or non-numeric for all transaction rows).")
            else:
                # More informative message if OB parsing failed
                ob_value_to_print = str(opening_balance_row[bal_col].iloc[0]) if bal_col in opening_balance_row.columns and not opening_balance_row.empty else "<OB Balance Missing/Empty>"
                print(f"    WARN: Could not parse a valid numeric opening balance value from '{ob_value_to_print}' in the separated OB row's '{bal_col}' column. Skipping recalculation.")

        except Exception as e:
            print(f"    ERROR during balance recalculation: {e}. Skipping.")
            print(traceback.format_exc()) # Print full traceback for debugging
            # Clean up partial calculation column if error occurred mid-way
            if calculated_balance_col in df_processed.columns:
                 df_processed.drop(columns=[calculated_balance_col], inplace=True)
            calculated_balance_added = False # Ensure flag is reset

    # Handle cases where recalculation wasn't attempted
    elif opening_balance_row is None: print("     Skipping balance recalculation: Opening Balance row not found or separated.")
    elif df_processed.empty: print("     Skipping balance recalculation: No transaction rows to calculate on.")
    else: print(f"     Skipping balance recalculation: Prerequisites not met (e.g., missing amount columns or parsable OB). Check previous steps.")


    # --- Step 9: Re-attach Opening Balance Row ---
    print("  9. Handling Opening Balance row concatenation...")
    if opening_balance_row is not None:
         print("     Attempting to reintegrate Opening Balance row...")
         # Align columns - add missing columns to OB row with appropriate defaults
         all_cols = df_processed.columns.tolist() # Get columns from the processed transactions
         if calculated_balance_added and calculated_balance_col not in all_cols:
             # Should not happen if added correctly, but safety check
              all_cols.append(calculated_balance_col)

         for col in all_cols:
             if col not in opening_balance_row.columns:
                 if col in ['year', 'month', 'day', 'day_of_week']: opening_balance_row[col] = pd.NA
                 elif col == calculated_balance_col:
                     # Assign the parsed OB value if available, else NaN
                     opening_balance_row[col] = opening_balance_numeric if pd.notna(opening_balance_numeric) else np.nan
                 elif col == 'transaction_type': opening_balance_row[col] = 'Opening Balance'
                 elif col in [wd_amt_col, dp_amt_col]: opening_balance_row[col] = 0 # OB has 0 withdrawal/deposit
                 # For original balance column, keep its value (or NaN if it was missing/unparsable)
                 elif col == bal_col:
                     if bal_col not in opening_balance_row.columns: # If balance col itself was missing in OB row
                         opening_balance_row[col] = opening_balance_numeric if pd.notna(opening_balance_numeric) else np.nan
                     # else: it already exists, keep its value (parsed or NaN)
                 else: opening_balance_row[col] = np.nan # Default for others like text fields if not present


         # Ensure OB row has correct types matching df_processed where possible, BEFORE concatenation
         # Dates (convert existing date cols in OB row to datetime)
         for col in [date_col, val_dt_col]:
            if col in opening_balance_row.columns:
                opening_balance_row[col] = pd.to_datetime(opening_balance_row[col], errors='coerce')

         # Amounts (ensure numeric, apply same cleaning logic as before)
         cols_to_num = [wd_amt_col, dp_amt_col, bal_col] + ([calculated_balance_col] if calculated_balance_col in opening_balance_row.columns else [])
         for col in cols_to_num:
             if col in opening_balance_row.columns:
                 try:
                     # Apply same cleaning as main data for consistency before numeric conversion
                     ob_val_str = str(opening_balance_row[col].iloc[0]).strip().replace(',', '')
                     ob_val_str = re.sub(r'\s*(Cr|Dr)\.?$', '', ob_val_str)
                     opening_balance_row[col] = pd.to_numeric(ob_val_str, errors='coerce')
                 except:
                     opening_balance_row[col] = np.nan # Fallback if conversion fails

         # Text columns (ensure string, apply placeholder logic if needed)
         for col in text_cols_to_process: # [narr_col, ref_col]
            if col in opening_balance_row.columns:
                try:
                    text_val = str(opening_balance_row[col].iloc[0]).strip()
                    null_like_vals = ['nan', 'none', 'null', 'na', 'n/a', '#n/a', '<na>', 'nat', '-', '']
                    # If text is null-like or NaN after stripping, set to a specific OB placeholder
                    if pd.isna(text_val) or text_val.lower() in null_like_vals:
                         fill_value = 'Opening Balance Narration' if col == narr_col else ('OB Ref' if col == ref_col else np.nan)
                         opening_balance_row[col] = fill_value
                    else:
                        opening_balance_row[col] = text_val # Keep original OB text if valid
                except:
                    opening_balance_row[col] = 'OB Text Error' # Fallback

         # --- Concatenation ---
         try:
             # If transaction df exists, align OB columns to it
             if not df_processed.empty:
                 cols_order = df_processed.columns
                 # Reindex OB row to match df_processed exactly, adding missing columns as NaN
                 opening_balance_row_aligned = opening_balance_row.reindex(columns=cols_order)
                 # Concatenate: OB row first, then transactions
                 df_processed = pd.concat([opening_balance_row_aligned, df_processed], ignore_index=True)
                 print("     Successfully concatenated Opening Balance row to the top.")
             else:
                 # If only OB row exists, make it the main df
                 # Ensure it has at least the final required columns before proceeding
                 essential_cols = list(FINAL_BANK_COLS)
                 for col in essential_cols:
                     if col not in opening_balance_row.columns:
                         opening_balance_row[col] = np.nan # Add missing essential columns as NaN
                 # Use the OB row as the processed df
                 df_processed = opening_balance_row
                 print("     Using Opening Balance row as the main DataFrame (no transaction rows found).")

         except Exception as e:
             print(f"     WARN: Could not reliably concat opening balance row: {e}. OB Row might be lost or misaligned.")
             print(traceback.format_exc())
             # Fallback: If concatenation failed, try to proceed without OB row if transactions exist
             if opening_balance_row is not None and not df_processed.empty:
                 print("     Proceeding with transaction data only.")
             elif opening_balance_row is not None: # Only OB row existed and concat failed
                 print("     Cannot proceed as concatenation failed and no other data exists.")
                 return None # Critical error
    else:
        print("     No Opening Balance row was separated, nothing to concatenate.")


    # --- Step 10: Final Sort ---
    print("  10. Final sorting of data (including OB row if present)...")
    if date_col in df_processed.columns and pd.api.types.is_datetime64_any_dtype(df_processed[date_col]):
        # Sort by Date (NaNs/OB first), potentially add secondary sort keys if needed
        sort_keys = [date_col]
        # If Value Dt is reliable and present, use it as secondary sort
        # if val_dt_col in df_processed.columns and pd.api.types.is_datetime64_any_dtype(df_processed[val_dt_col]):
        #    sort_keys.append(val_dt_col)
        print(f"      Sorting by {sort_keys} (NaNs/OB first using mergesort)...")
        df_processed.sort_values(by=sort_keys, inplace=True, na_position='first', kind='mergesort')
        print(f"      Sorted by '{date_col}' (NaNs/OB first).")
    else:
        # Check if OB row is the only row and date is missing
        if len(df_processed) == 1 and opening_balance_row is not None and pd.isna(df_processed[date_col].iloc[0]):
             print(f"      Skipping sort: Only Opening Balance row exists with no valid date.")
        elif date_col not in df_processed.columns:
            print(f"      Cannot sort: Date column '{date_col}' is missing.")
        else:
             print(f"      Cannot sort: Date column '{date_col}' is not in datetime format after processing.")


    # --- Step 11: Select Final Columns ---
    print(f"  11. Selecting final columns: {FINAL_BANK_COLS}")
    # Check which of the desired final columns actually exist in the df *after* processing
    cols_to_keep = [col for col in FINAL_BANK_COLS if col in df_processed.columns]
    cols_missing_from_final = [col for col in FINAL_BANK_COLS if col not in df_processed.columns]

    if cols_missing_from_final:
        print(f"    WARN: The following desired final columns were not found in the processed data and will be omitted: {cols_missing_from_final}")

    if not cols_to_keep:
        print(f"    ERROR: None of the desired final columns {FINAL_BANK_COLS} were found in the processed DataFrame. Cannot create final output.")
        return None

    # Create the final DataFrame with only the selected columns
    df_final = df_processed[cols_to_keep].copy()
    print(f"    Selected {len(df_final.columns)} columns for final output: {list(df_final.columns)}")


    # --- Step 12: Final Inspection ---
    print("  12. Final Inspection...")
    print("      ------------------------------")
    print("      Final Bank Statement DataFrame Info (Selected Columns):")
    buffer = StringIO()
    df_final.info(memory_usage='deep', buf=buffer)
    print(buffer.getvalue())
    print("\n      Missing values after processing (Selected Columns):")
    missing_summary = df_final.isnull().sum()
    missing_summary = missing_summary[missing_summary > 0]
    if not missing_summary.empty:
        print(missing_summary.to_string())
    else:
        print("      No missing values remaining in final selected columns.")
    print(f"\n      Final Rows: {len(df_final)}")
    print("      ------------------------------")
    print(f"--- Finished Bank Statement Preprocessing: {filename} ---")

    return df_final


# =============================================================================
# 4. MAIN PROCESSING LOOP (BANK ONLY)
# =============================================================================
print("\n--- 4. Starting Bank Statement File Processing Loop ---")

processed_bank_count = 0
error_bank_count = 0

if not os.path.exists(bank_input_dir):
    print(f"ERROR: Bank input directory not found: {os.path.abspath(bank_input_dir)}. Please check the path and ensure Drive is mounted if using Colab.")
else:
    print(f"Reading Bank Statement files from: {os.path.abspath(bank_input_dir)}")
    try:
        all_items_in_dir = os.listdir(bank_input_dir)
        print(f"    Items found by os.listdir: {len(all_items_in_dir)}")
        # Filter more robustly for CSV
        all_bank_files = [item for item in all_items_in_dir if item.lower().endswith('.csv') and not item.startswith('.')]
        print(f"     Found {len(all_bank_files)} CSV files to process.")

        if not all_bank_files:
             print("     WARN: No CSV files found in the Bank input directory.")

        for filename in all_bank_files:
            input_path = os.path.join(bank_input_dir, filename)
            # Construct output filename robustly
            base_name, _ = os.path.splitext(filename)
            # Using a distinct suffix for this version
            output_filename = f"{base_name}_preprocessed_final_cols.csv"
            output_path = os.path.join(bank_output_dir, output_filename)

            try:
                print(f"\n>>> Reading Bank Statement file: {filename}")
                # Try reading with common encodings
                df_raw = None
                try: df_raw = pd.read_csv(input_path, encoding='utf-8')
                except UnicodeDecodeError:
                    print("      UTF-8 decoding failed, trying latin-1...")
                    try: df_raw = pd.read_csv(input_path, encoding='latin1')
                    except UnicodeDecodeError:
                        print("      latin-1 decoding failed, trying cp1252...")
                        try: df_raw = pd.read_csv(input_path, encoding='cp1252')
                        except UnicodeDecodeError as ude:
                             print(f"  ERROR: Could not decode file {filename} with utf-8, latin1, or cp1252. Skipping. Error: {ude}"); error_bank_count += 1
                             continue # Skip to next file
                except pd.errors.EmptyDataError:
                    print(f"  WARN: File {filename} is empty. Skipping."); error_bank_count += 1
                    continue # Skip to next file
                except FileNotFoundError:
                    print(f"  ERROR: File not found at {input_path}. Skipping."); error_bank_count += 1
                    continue # Skip to next file
                except Exception as read_err: # Catch other potential read errors
                    print(f"  ERROR: Failed to read CSV {filename}. Error: {read_err}")
                    print(traceback.format_exc()); error_bank_count += 1
                    continue

                if df_raw is None:
                    print(f"  ERROR: DataFrame df_raw is None after read attempts for {filename}. Skipping."); error_bank_count += 1
                    continue

                # --- Call Bank Preprocessing Function ---
                df_clean = preprocess_bank_statement_final_cols(df_raw, filename)

                if df_clean is not None and not df_clean.empty:
                    print(f"  --> Saving preprocessed Bank Statement file to: {output_path}")
                    df_clean.to_csv(output_path, index=False, encoding='utf-8')
                    processed_bank_count += 1
                elif df_clean is None:
                    print(f"  --> Skipping save for {filename} due to critical processing error (check logs above).")
                    error_bank_count += 1
                else: # df_clean is empty
                    print(f"  --> Skipping save for {filename} because the resulting DataFrame is empty after cleaning/selection.")
                    error_bank_count += 1

            except Exception as e: # Catch errors during processing or saving
                print(f"  ERROR: An unexpected error occurred while processing Bank file {filename}: {e}")
                print(traceback.format_exc()); error_bank_count += 1

    except FileNotFoundError:
         print(f"ERROR: Bank input directory itself not found during listdir: {bank_input_dir}")
         error_bank_count = -1 # Indicate directory level error
    except Exception as list_err:
        print(f"ERROR: Could not list or process Bank directory contents for {bank_input_dir}: {list_err}"); error_bank_count = -1

print(f"\nBank Statement Processing Summary: {processed_bank_count} files succeeded, {error_bank_count} files failed/skipped.")


# =============================================================================
# 5. FINAL SCRIPT COMPLETION MESSAGE (BANK)
# =============================================================================
print("\n" + "="*30 + " Bank Script Finished " + "="*30)
print("--- All Bank Statement preprocessing tasks complete! ---")
print(f"   Preprocessed Bank Statement files (final columns) saved in: {os.path.abspath(bank_output_dir)}")
print("="*70)

--- 1. Loading Libraries ---
--- 2. Setting Bank Statement Configuration ---
     Creating Bank output directory (if needed): /content/drive/MyDrive/Capstone Preprocessing/B2C/bank_preprocessed1
     Bank Input Directory: /content/drive/MyDrive/Capstone Preprocessing/B2C/bank_statements1
     Bank Output Directory: /content/drive/MyDrive/Capstone Preprocessing/B2C/bank_preprocessed1
     Required Bank Columns: ['Date', 'Narration', 'Chq./Ref.No.', 'Value Dt', 'Withdrawal Amt.', 'Deposit Amt.', 'Closing Balance']
     Desired Final Bank Columns: ['Date', 'Narration', 'Chq./Ref.No.', 'Value Dt', 'Withdrawal Amt.', 'Deposit Amt.', 'Closing Balance']

--- 3. Defining Bank Statement Preprocessing Function ---

--- 4. Starting Bank Statement File Processing Loop ---
Reading Bank Statement files from: /content/drive/MyDrive/Capstone Preprocessing/B2C/bank_statements1
    Items found by os.listdir: 100
     Found 100 CSV files to process.

>>> Reading Bank Statement file: 0L725_Bank_Statement.

<ipython-input-8-1911b4f47f16>:162: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_processed[col].replace(['-', 'nil', 'null', 'N/A', 'NA', 'NaN', 'NaT', ''], np.nan, inplace=True)
<ipython-input-8-1911b4f47f16>:162: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when

       NaNs before numeric conversion: 35606, After conversion: 35606
     Processing 'Deposit Amt.' column (converting from string)...
       NaNs before numeric conversion: 573, After conversion: 573
     Processing 'Closing Balance' column (converting from string)...
       NaNs before numeric conversion: 0, After conversion: 0
     Cleaning 'Narration' column...
       No common placeholder values found to replace in 'Narration'.
     Cleaning 'Chq./Ref.No.' column...
       No common placeholder values found to replace in 'Chq./Ref.No.'.
  4. Checking uniqueness of 'Chq./Ref.No.'...


<ipython-input-8-1911b4f47f16>:211: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_processed[col].replace(['', '-', 'nil', 'null', 'N/A', 'NA', 'NaN', 'NaT'], np.nan, inplace=True)
<ipython-input-8-1911b4f47f16>:284: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when

    INFO: All non-missing values in 'Chq./Ref.No.' appear unique after cleaning.
  5. Handling remaining missing values...
     Filling 35606 missing 'Withdrawal Amt.' values with 0.
     Filling 573 missing 'Deposit Amt.' values with 0.
  6. Checking for duplicate transaction rows...
     No duplicate transaction rows found based on key columns: ['Date', 'Narration', 'Withdrawal Amt.', 'Deposit Amt.', 'Chq./Ref.No.'].
  7. Adding temporary features (date components, transaction type)...
     Added temporary date features.
     Added temporary transaction type.
  8. Attempting to recalculate closing balance (into temporary column)...
     Using parsed Opening Balance: 62607.39
       Sorting by ['Date'] for calculation...
     Successfully added temporary 'calculated_balance' column.
     Comparison with original 'Closing Balance' (36179 comparable rows): Max diff: 0.00, Mean diff: 0.00
       Calculated balance matches original balance (within 0.01 tolerance) where comparable.
  9. Ha

...

       NaNs before numeric conversion: 12869, After conversion: 12869
     Processing 'Deposit Amt.' column (converting from string)...
       NaNs before numeric conversion: 782, After conversion: 782
     Processing 'Closing Balance' column (converting from string)...
       NaNs before numeric conversion: 0, After conversion: 0
     Cleaning 'Narration' column...


<ipython-input-8-1911b4f47f16>:211: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_processed[col].replace(['', '-', 'nil', 'null', 'N/A', 'NA', 'NaN', 'NaT'], np.nan, inplace=True)
<ipython-input-8-1911b4f47f16>:284: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when

       No common placeholder values found to replace in 'Narration'.
     Cleaning 'Chq./Ref.No.' column...
       No common placeholder values found to replace in 'Chq./Ref.No.'.
  4. Checking uniqueness of 'Chq./Ref.No.'...
    INFO: All non-missing values in 'Chq./Ref.No.' appear unique after cleaning.
  5. Handling remaining missing values...
     Filling 12869 missing 'Withdrawal Amt.' values with 0.
     Filling 782 missing 'Deposit Amt.' values with 0.
  6. Checking for duplicate transaction rows...
     No duplicate transaction rows found based on key columns: ['Date', 'Narration', 'Withdrawal Amt.', 'Deposit Amt.', 'Chq./Ref.No.'].
  7. Adding temporary features (date components, transaction type)...
     Added temporary date features.
     Added temporary transaction type.
  8. Attempting to recalculate closing balance (into temporary column)...
     Using parsed Opening Balance: 85710.47
       Sorting by ['Date'] for calculation...
     Successfully added temporary 'calcula

<ipython-input-8-1911b4f47f16>:490: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  opening_balance_row[col] = pd.to_datetime(opening_balance_row[col], errors='coerce')
<ipython-input-8-1911b4f47f16>:490: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  opening_balance_row[col] = pd.to_datetime(opening_balance_row[col], errors='coerce')



Bank Statement Processing Summary: 100 files succeeded, 0 files failed/skipped.

============================== Bank Script Finished ==============================
--- All Bank Statement preprocessing tasks complete! ---
   Preprocessed Bank Statement files (final columns) saved in: /content/drive/MyDrive/Capstone Preprocessing/B2C/bank_preprocessed1


In [ ]:
# =============================================================================
# SCRIPT: Bank Statement Preprocessing
# PURPOSE: Reads raw bank statement CSV files, preprocesses them, selects
#          final columns, and saves the cleaned data to a separate
#          output directory.
# =============================================================================

# =============================================================================
# 1. IMPORTS
# =============================================================================
print("--- 1. Loading Libraries ---")
import os
import pandas as pd
import numpy as np
import re # Useful for cleaning values
from io import StringIO # For capturing df.info() output
import traceback # For detailed error printing

# =============================================================================
# 2. CONFIGURATION (BANK SPECIFIC)
# =============================================================================
print("--- 2. Setting Bank Statement Configuration ---")

# --- Input/Output Directories ---
# Adjust these paths if not using Google Drive or if your structure differs
BASE_DIR = '/content/drive/MyDrive/Capstone Preprocessing/B2C' # Example base if using Colab
bank_input_dir = os.path.join(BASE_DIR, 'bank_statements1')
bank_output_dir = os.path.join(BASE_DIR, 'bank_preprocessed1')

# --- Create Bank Output Directory ---
print(f"     Creating Bank output directory (if needed): {os.path.abspath(bank_output_dir)}")
os.makedirs(bank_output_dir, exist_ok=True)

# --- Bank Statement Column Names ---
# !!! CHECK THESE REQUIRED BANK NAMES AGAINST YOUR ACTUAL FILE HEADER !!!
REQUIRED_BANK_COLS = ['Date', 'Narration', 'Chq./Ref.No.', 'Value Dt',
                       'Withdrawal Amt.', 'Deposit Amt.', 'Closing Balance']
# --- Desired FINAL Bank Columns ---
FINAL_BANK_COLS = ['Date', 'Narration', 'Chq./Ref.No.', 'Value Dt',
                   'Withdrawal Amt.', 'Deposit Amt.', 'Closing Balance']


print(f"     Bank Input Directory: {os.path.abspath(bank_input_dir)}")
print(f"     Bank Output Directory: {os.path.abspath(bank_output_dir)}")
print(f"     Required Bank Columns: {REQUIRED_BANK_COLS}") # Check these!
print(f"     Desired Final Bank Columns: {FINAL_BANK_COLS}") # Check these!


# =============================================================================
# 3. BANK STATEMENT PREPROCESSING FUNCTION
# =============================================================================
print("\n--- 3. Defining Bank Statement Preprocessing Function ---")

def preprocess_bank_statement_final_cols(df, filename=""):
    """
    Preprocesses a bank statement DataFrame and keeps only specified final columns.

    Performs the following steps:
    1. Verifies required columns (as defined in REQUIRED_BANK_COLS) exist. *** Check REQUIRED_BANK_COLS ***
    2. Separates a potential 'Opening Balance' row.
    3. Cleans and converts date columns ('Date', 'Value Dt') to datetime (DD-MM-YYYY format). Drops rows with invalid primary 'Date'.
    4. Cleans and converts amount columns ('Withdrawal Amt.', 'Deposit Amt.', 'Closing Balance') to numeric.
    5. Cleans text columns ('Narration', 'Chq./Ref.No.'), strips whitespace, handles NaNs.
    6. Checks for uniqueness of 'Chq./Ref.No.'.
    7. Handles missing values (fills 'Value Dt', amounts with 0, text with placeholders).
    8. Checks for and removes duplicate transaction rows.
    9. Adds temporary features: 'year', 'month', 'day', 'day_of_week', 'transaction_type'.
    10. Attempts to recalculate 'Closing Balance' into a temporary column.
    11. Reattaches the Opening Balance row.
    12. Sorts the DataFrame by 'Date'.
    13. Selects only the columns specified in FINAL_BANK_COLS.
    14. Performs a final inspection.

    Args:
        df (pd.DataFrame): The raw bank statement DataFrame.
        filename (str, optional): Original filename for logging purposes. Defaults to "".

    Returns:
        pd.DataFrame or None: The preprocessed DataFrame containing only FINAL_BANK_COLS,
                              or None if critical errors occur.
    """
    print(f"\n--- Starting Bank Statement Preprocessing for: {filename} ---")
    # --- Initial Checks ---
    if df.empty:
        print("  WARN: Input DataFrame is empty. Skipping.")
        return None

    df_processed = df.copy()
    print(f"     Initial raw columns: {df_processed.columns.tolist()}")

    # --- Step 1: Verify Required Columns ---
    print("  1. Verifying required columns...")
    actual_cols = df_processed.columns.tolist()
    # Trim whitespace from actual column names for comparison
    actual_cols_trimmed = {col.strip(): col for col in actual_cols}
    df_processed.columns = [col.strip() for col in actual_cols] # Apply trimming
    actual_cols = df_processed.columns.tolist() # Update list

    missing_req = [col for col in REQUIRED_BANK_COLS if col not in actual_cols_trimmed]
    if missing_req:
        print(f"  ERROR: Missing REQUIRED Bank columns: {missing_req}. Required are {REQUIRED_BANK_COLS}. Found columns: {actual_cols}. Cannot proceed. Skipping file.")
        return None
    print(f"     Found required Bank columns: {REQUIRED_BANK_COLS}")

    # Use potentially trimmed names
    date_col = 'Date'
    narr_col = 'Narration'
    ref_col = 'Chq./Ref.No.'
    val_dt_col = 'Value Dt'
    wd_amt_col = 'Withdrawal Amt.'
    dp_amt_col = 'Deposit Amt.'
    bal_col = 'Closing Balance'

    # --- Step 2: Handle Opening Balance Row ---
    print("  2. Checking for and separating Opening Balance row...")
    opening_balance_row = None
    if narr_col in df_processed.columns and not df_processed.empty:
        # Ensure the first narration value is treated as a string before processing
        first_narration_val = df_processed[narr_col].iloc[0]
        first_narration = str(first_narration_val).strip().lower() if pd.notna(first_narration_val) else ""

        # More robust check for opening balance narration
        ob_keywords = ['opening balance', 'o/b', 'opng bal', 'op balance', 'brought forward', 'bal b/f']
        if any(keyword in first_narration for keyword in ob_keywords):
             # Check if Date is null or if BOTH Withdrawal and Deposit are null/zero
             first_date_is_null = pd.isna(df_processed[date_col].iloc[0]) if date_col in df_processed.columns else True

             # Check amounts carefully - handle cases where columns might not exist yet
             wd_val = 0
             dp_val = 0
             if wd_amt_col in df_processed.columns and pd.notna(df_processed[wd_amt_col].iloc[0]):
                 wd_val = pd.to_numeric(str(df_processed[wd_amt_col].iloc[0]).replace(',','').strip(), errors='coerce')
                 wd_val = wd_val if pd.notna(wd_val) else 0 # Treat parsing failure as 0 for this check
             if dp_amt_col in df_processed.columns and pd.notna(df_processed[dp_amt_col].iloc[0]):
                 dp_val = pd.to_numeric(str(df_processed[dp_amt_col].iloc[0]).replace(',','').strip(), errors='coerce')
                 dp_val = dp_val if pd.notna(dp_val) else 0 # Treat parsing failure as 0 for this check

             first_wd_dp_are_zero_or_null = (wd_val == 0 and dp_val == 0)

             # Separate if date is null OR wd/dp are zero/null AND narration matches keywords
             if first_date_is_null or first_wd_dp_are_zero_or_null:
                 opening_balance_row = df_processed.iloc[0:1].copy()
                 df_processed = df_processed.iloc[1:].copy().reset_index(drop=True)
                 print(f"     Detected and separated potential Opening Balance row based on Narration ('{first_narration}') and missing Date or zero/missing Amounts.")
             else:
                 print(f"     First row narration ('{first_narration}') contains keywords, but Date is present and Amounts are non-zero. Not separating automatically.")
        else:
            print(f"     No distinct Opening Balance pattern detected in first row Narration ('{first_narration}').")

    # --- Step 3: Handle Data Types and Cleaning ---
    print("  3. Processing Data Types & Cleaning...")
    # Process Date Columns
    date_cols_to_process = [date_col, val_dt_col]
    for col in date_cols_to_process:
        if col in df_processed.columns:
            print(f"     Processing '{col}' column...")
            original_nan_count = df_processed[col].isnull().sum()
            # Convert to string first
            df_processed[col] = df_processed[col].astype(str).str.strip()
            df_processed[col].replace(['-', 'nil', 'null', 'N/A', 'NA', 'NaN', 'NaT', ''], np.nan, inplace=True)

            # Try specific format first (DD-MM-YYYY)
            dates_parsed = pd.to_datetime(df_processed[col], format='%d-%m-%Y', errors='coerce')
            invalid_mask_fmt = dates_parsed.isna()
            print(f"       Invalid dates after DD-MM-YYYY format: {invalid_mask_fmt.sum()}")

            # Try generic parser for those that failed the specific format
            if invalid_mask_fmt.any():
                 print(f"       Trying generic date parser (dayfirst=True) for {invalid_mask_fmt.sum()} failed dates...")
                 # Apply generic only where specific format failed
                 dates_generic = pd.to_datetime(df_processed.loc[invalid_mask_fmt, col], errors='coerce', dayfirst=True)
                 dates_parsed.loc[invalid_mask_fmt] = dates_generic # Update main series
                 print(f"       Invalid dates remaining after generic pass: {dates_parsed.isna().sum()}")

            df_processed[col] = dates_parsed # Assign back

            converted_nan_count = df_processed[col].isnull().sum()
            # Note: NaNs could decrease if placeholder strings were converted to NaT then parsed correctly later
            print(f"       NaNs/NaTs before parsing: {original_nan_count}, After parsing: {converted_nan_count}")

    # Drop rows if primary 'Date' is invalid AFTER parsing attempts
    rows_before_drop = len(df_processed)
    if date_col in df_processed.columns:
        df_processed.dropna(subset=[date_col], inplace=True)
        rows_removed = rows_before_drop - len(df_processed)
        if rows_removed > 0:
             print(f"       Removed {rows_removed} rows with invalid/missing primary '{date_col}' after parsing attempts.")
    else:
         print(f"     WARN: Cannot drop rows based on missing dates as '{date_col}' column not found.")

    # Check if only OB row remains or no data
    if df_processed.empty and opening_balance_row is None:
        print(f"  ERROR: No valid transaction data remaining after cleaning '{date_col}'. Skipping file.")
        return None
    elif df_processed.empty and opening_balance_row is not None:
        print(f"  WARN: Only Opening Balance row remains after cleaning '{date_col}'. Proceeding with OB row only.")
        # Ensure required cols exist in OB row if it's the only thing left
        for req_col in REQUIRED_BANK_COLS:
             if req_col not in opening_balance_row.columns: opening_balance_row[req_col] = np.nan

    # Process Amount Columns
    amount_cols_to_process = [wd_amt_col, dp_amt_col, bal_col]
    for col in amount_cols_to_process:
        if col in df_processed.columns:
            print(f"     Processing '{col}' column (converting from string)...")
            original_nan_count = df_processed[col].isnull().sum()
            df_processed[col] = df_processed[col].astype(str).str.strip()
            # Handle blanks, hyphens, common text placeholders
            df_processed[col].replace(['', '-', 'nil', 'null', 'N/A', 'NA', 'NaN', 'NaT'], np.nan, inplace=True)
            # Remove commas used as thousand separators
            df_processed[col] = df_processed[col].str.replace(',', '', regex=False)
            # Specific check for credit notations like ' Cr' or ' Dr' (remove them) - adjust if needed
            df_processed[col] = df_processed[col].str.replace(r'\s*(Cr|Dr)\.?$', '', regex=True)

            df_processed[col] = pd.to_numeric(df_processed[col], errors='coerce')
            converted_nan_count = df_processed[col].isnull().sum()
            # Note: NaNs could decrease if placeholder strings were converted to NaN then parsed correctly
            print(f"       NaNs before numeric conversion: {original_nan_count}, After conversion: {converted_nan_count}")
        elif col in REQUIRED_BANK_COLS: # Check if a required amount col is missing entirely
            print(f"     WARN: Required amount column '{col}' not found in DataFrame. Will be missing from output.")


    # Process Text Columns
    text_cols_to_process = [narr_col, ref_col]
    for col in text_cols_to_process:
        if col in df_processed.columns:
            print(f"     Cleaning '{col}' column...")
            # Convert to string, strip whitespace
            df_processed[col] = df_processed[col].astype(str).str.strip()
            # Replace specific placeholder strings (case-insensitive check then replace)
            null_like_vals = ['nan', 'none', 'null', 'na', 'n/a', '#n/a', '<na>', 'nat', '-', '']
            mask = df_processed[col].str.lower().isin(null_like_vals) | df_processed[col].isnull()
            # Count how many will be replaced before doing it
            num_to_replace = mask.sum()
            if num_to_replace > 0:
                 print(f"       Replacing {num_to_replace} placeholder/null values in '{col}' with NaN.")
                 df_processed.loc[mask, col] = np.nan
            else:
                 print(f"       No common placeholder values found to replace in '{col}'.")
        elif col in REQUIRED_BANK_COLS:
             print(f"     WARN: Required text column '{col}' not found in DataFrame. Will be missing from output.")


    # --- Step 4: Check Uniqueness of Reference Numbers ---
    print(f"  4. Checking uniqueness of '{ref_col}'...")
    if ref_col in df_processed.columns:
        # Check uniqueness *after* cleaning, excluding NaNs resulting from cleaning
        refs_to_check = df_processed[ref_col].dropna().astype(str)

        if not refs_to_check.empty:
            ref_counts = refs_to_check.value_counts()
            non_unique_refs = ref_counts[ref_counts > 1]
            if not non_unique_refs.empty:
                print(f"    WARN: Found {len(non_unique_refs)} distinct reference numbers that are NOT unique (totaling {non_unique_refs.sum()} occurrences) in '{ref_col}' after cleaning.")
                # print(f"      Examples (top 5):\n{non_unique_refs.head(5).to_string()}") # Optional: print examples
            else:
                print(f"    INFO: All non-missing values in '{ref_col}' appear unique after cleaning.")
        else:
            print(f"    INFO: '{ref_col}' column contains only missing values after cleaning.")
    else:
        print(f"    INFO: Column '{ref_col}' not found, skipping uniqueness check.")


    # --- Step 5: Handle Missing Values (Post Cleaning) ---
    print("  5. Handling remaining missing values...")
    # Fill missing 'Value Dt' with 'Date'
    if val_dt_col in df_processed.columns and date_col in df_processed.columns:
        missing_value_dt = df_processed[val_dt_col].isnull().sum()
        if missing_value_dt > 0:
            if pd.api.types.is_datetime64_any_dtype(df_processed[date_col]):
                 print(f"     Filling {missing_value_dt} missing '{val_dt_col}' with corresponding '{date_col}'.")
                 df_processed[val_dt_col].fillna(df_processed[date_col], inplace=True)
            else:
                 print(f"     WARN: Cannot fill missing '{val_dt_col}' because '{date_col}' is not a valid datetime column after cleaning. NaNs remain.")

    # Fill missing Amounts (Withdrawal/Deposit) with 0 (after numeric conversion)
    for col in [wd_amt_col, dp_amt_col]:
        if col in df_processed.columns:
             missing_amount = df_processed[col].isnull().sum()
             if missing_amount > 0:
                 print(f"     Filling {missing_amount} missing '{col}' values with 0.")
                 df_processed[col].fillna(0, inplace=True)

    # Warn about missing 'Closing Balance' - DON'T fill with 0 unless calculated later
    if bal_col in df_processed.columns:
        missing_balance = df_processed[bal_col].isnull().sum()
        if missing_balance > 0:
             # Changed from WARN to INFO as recalculation might fix it
             print(f"     INFO: {missing_balance} rows have missing '{bal_col}' after conversion. Will remain NaN unless calculation succeeds.")

    # Fill missing text fields (Narration/RefNo) with placeholders *after* cleaning
    for col in [narr_col, ref_col]:
         if col in df_processed.columns:
             missing_text = df_processed[col].isnull().sum() # Check NaNs again
             if missing_text > 0:
                 fill_value = 'Unknown Narration' if col == narr_col else 'No Ref'
                 print(f"     Filling {missing_text} remaining missing '{col}' with '{fill_value}'.")
                 df_processed[col].fillna(fill_value, inplace=True)


    # --- Step 6: Handle Duplicate Transaction Rows ---
    print("  6. Checking for duplicate transaction rows...")
    # Use required cols for duplicate check as they should always exist and be filled
    key_cols_for_dup_check = [date_col, narr_col, wd_amt_col, dp_amt_col, ref_col]
    existing_key_cols = [c for c in key_cols_for_dup_check if c in df_processed.columns]

    if len(existing_key_cols) >= 4 and not df_processed.empty: # Require at least Date + Narration + Amounts for meaningful check
        duplicates_count = df_processed.duplicated(subset=existing_key_cols).sum()
        if duplicates_count > 0:
            print(f"     Found and removing {duplicates_count} duplicate transaction rows based on {existing_key_cols}.")
            df_processed.drop_duplicates(subset=existing_key_cols, keep='first', inplace=True)
        else:
            print(f"     No duplicate transaction rows found based on key columns: {existing_key_cols}.")
    elif df_processed.empty:
        print("     Skipping duplicate check as DataFrame is empty (excluding potential OB row).")
    else:
        print(f"     Skipping duplicate check as not enough key columns ({existing_key_cols}) are available.")


    # --- Step 7: Feature Engineering (Temporary) ---
    print("  7. Adding temporary features (date components, transaction type)...")
    calculated_balance_col = 'calculated_balance' # Define name for potential new column
    # Initialize columns first
    df_processed['year'], df_processed['month'], df_processed['day'], df_processed['day_of_week'] = [pd.NA] * 4
    df_processed['transaction_type'] = 'Unknown' # Initialize

    if not df_processed.empty: # Only proceed if there are transaction rows
        # Add Date Features
        if date_col in df_processed.columns and pd.api.types.is_datetime64_any_dtype(df_processed[date_col]):
            df_processed['year'] = df_processed[date_col].dt.year
            df_processed['month'] = df_processed[date_col].dt.month
            df_processed['day'] = df_processed[date_col].dt.day
            df_processed['day_of_week'] = df_processed[date_col].dt.dayofweek # Monday=0, Sunday=6
            print("     Added temporary date features.")
        else:
            print(f"    WARN: Cannot extract date features because '{date_col}' is not a valid datetime column.")

        # Add Transaction Type
        # Use already cleaned/filled numeric amounts (filled with 0 in Step 5)
        wd_present = wd_amt_col in df_processed.columns
        dp_present = dp_amt_col in df_processed.columns

        # Ensure columns exist before accessing
        wd_num = df_processed[wd_amt_col] if wd_present else pd.Series(0, index=df_processed.index)
        dp_num = df_processed[dp_amt_col] if dp_present else pd.Series(0, index=df_processed.index)

        # Determine transaction type based on non-zero amounts
        df_processed.loc[wd_num > 0, 'transaction_type'] = 'Debit'
        df_processed.loc[dp_num > 0, 'transaction_type'] = 'Credit'
        # Handle cases where both might be zero (e.g., balance adjustments, informational lines)
        df_processed.loc[(wd_num == 0) & (dp_num == 0), 'transaction_type'] = 'Zero Amount'
        print("     Added temporary transaction type.")
        # Optional: Print distribution for verification
        # print(f"      Transaction Type distribution:\n{df_processed['transaction_type'].value_counts().to_string()}")
    else:
        print("     Skipping temporary feature engineering as DataFrame (excluding OB row) is empty.")


    # --- Step 8: Recalculate Closing Balance (Optional, creates temporary column) ---
    print("  8. Attempting to recalculate closing balance (into temporary column)...")
    calculated_balance_added = False
    opening_balance_numeric = np.nan

    # Prerequisites: OB row exists, balance col in OB, WD/DP cols in main df, main df not empty
    if opening_balance_row is not None and \
       bal_col in opening_balance_row.columns and \
       wd_amt_col in df_processed.columns and \
       dp_amt_col in df_processed.columns and \
       not df_processed.empty:

        try:
            # Try to parse OB from the separated row (apply same cleaning as main df)
            ob_val_str = str(opening_balance_row[bal_col].iloc[0]).strip().replace(',', '')
            ob_val_str = re.sub(r'\s*(Cr|Dr)\.?$', '', ob_val_str) # Remove Cr/Dr
            opening_balance_numeric = pd.to_numeric(ob_val_str, errors='coerce')

            if pd.notna(opening_balance_numeric):
                 print(f"     Using parsed Opening Balance: {opening_balance_numeric:.2f}")

                 # Ensure sorting before calculation for correctness on same-day transactions
                 sort_cols = [date_col]
                 # Optional: Sort by Value Dt as secondary if reliable
                 # if val_dt_col in df_processed.columns and pd.api.types.is_datetime64_any_dtype(df_processed[val_dt_col]):
                 #    sort_cols.append(val_dt_col)

                 # Add temporary index for stable sort if original index isn't unique or reliable, then remove
                 temp_index_needed = not df_processed.index.is_unique
                 if temp_index_needed:
                     df_processed['__temp_sort_idx__'] = range(len(df_processed))
                     sort_cols.append('__temp_sort_idx__')

                 if date_col in df_processed.columns and pd.api.types.is_datetime64_any_dtype(df_processed[date_col]):
                     print(f"       Sorting by {sort_cols} for calculation...")
                     # Use stable sort (mergesort)
                     df_processed = df_processed.sort_values(by=sort_cols, kind='mergesort', na_position='last')
                 else:
                     print("     WARN: Cannot sort reliably for balance calculation as primary date column is invalid.")

                 # Remove temporary index if added
                 if temp_index_needed:
                     df_processed.drop(columns=['__temp_sort_idx__'], inplace=True)

                 # Use already cleaned numeric columns (filled with 0 in Step 5)
                 deposit_numeric = df_processed[dp_amt_col]
                 withdrawal_numeric = df_processed[wd_amt_col]

                 # Calculate running balance
                 df_processed[calculated_balance_col] = opening_balance_numeric + \
                    (deposit_numeric - withdrawal_numeric).cumsum()
                 calculated_balance_added = True
                 print(f"     Successfully added temporary '{calculated_balance_col}' column.")

                 # Compare with original balance if it exists and is numeric
                 if bal_col in df_processed.columns:
                      original_bal_numeric = df_processed[bal_col] # Already converted to numeric or NaN
                      # Create comparison dataframe excluding rows where either is NaN
                      comparison_df = pd.DataFrame({
                          'original': original_bal_numeric,
                          'calculated': df_processed[calculated_balance_col]
                      }).dropna()

                      if not comparison_df.empty:
                          diff = (comparison_df['calculated'] - comparison_df['original']).abs()
                          tolerance = 0.01 # Define tolerance for float comparison
                          mismatches = diff[diff > tolerance]
                          print(f"     Comparison with original '{bal_col}' ({len(comparison_df)} comparable rows): Max diff: {diff.max():.2f}, Mean diff: {diff.mean():.2f}")
                          if not mismatches.empty:
                              print(f"       WARN: Found {len(mismatches)} rows where calculated balance differs (> {tolerance:.2f}) from original.")
                          else:
                              print(f"       Calculated balance matches original balance (within {tolerance:.2f} tolerance) where comparable.")

                          # Optionally: Consider replacing original balance with calculated if original is missing
                          # missing_orig_mask = df_processed[bal_col].isnull() & df_processed[calculated_balance_col].notnull()
                          # if missing_orig_mask.any():
                          #     print(f"       INFO: Filling {missing_orig_mask.sum()} missing original balances with calculated values.")
                          #     df_processed.loc[missing_orig_mask, bal_col] = df_processed.loc[missing_orig_mask, calculated_balance_col]
                      else:
                          print(f"     Cannot compare balances (original '{bal_col}' missing or non-numeric for all transaction rows).")
            else:
                # More informative message if OB parsing failed
                ob_value_to_print = str(opening_balance_row[bal_col].iloc[0]) if bal_col in opening_balance_row.columns and not opening_balance_row.empty else "<OB Balance Missing/Empty>"
                print(f"    WARN: Could not parse a valid numeric opening balance value from '{ob_value_to_print}' in the separated OB row's '{bal_col}' column. Skipping recalculation.")

        except Exception as e:
            print(f"    ERROR during balance recalculation: {e}. Skipping.")
            print(traceback.format_exc()) # Print full traceback for debugging
            # Clean up partial calculation column if error occurred mid-way
            if calculated_balance_col in df_processed.columns:
                 df_processed.drop(columns=[calculated_balance_col], inplace=True)
            calculated_balance_added = False # Ensure flag is reset

    # Handle cases where recalculation wasn't attempted
    elif opening_balance_row is None: print("     Skipping balance recalculation: Opening Balance row not found or separated.")
    elif df_processed.empty: print("     Skipping balance recalculation: No transaction rows to calculate on.")
    else: print(f"     Skipping balance recalculation: Prerequisites not met (e.g., missing amount columns or parsable OB). Check previous steps.")


    # --- Step 9: Re-attach Opening Balance Row ---
    print("  9. Handling Opening Balance row concatenation...")
    if opening_balance_row is not None:
         print("     Attempting to reintegrate Opening Balance row...")
         # Align columns - add missing columns to OB row with appropriate defaults
         all_cols = df_processed.columns.tolist() # Get columns from the processed transactions
         if calculated_balance_added and calculated_balance_col not in all_cols:
             # Should not happen if added correctly, but safety check
              all_cols.append(calculated_balance_col)

         for col in all_cols:
             if col not in opening_balance_row.columns:
                 if col in ['year', 'month', 'day', 'day_of_week']: opening_balance_row[col] = pd.NA
                 elif col == calculated_balance_col:
                     # Assign the parsed OB value if available, else NaN
                     opening_balance_row[col] = opening_balance_numeric if pd.notna(opening_balance_numeric) else np.nan
                 elif col == 'transaction_type': opening_balance_row[col] = 'Opening Balance'
                 elif col in [wd_amt_col, dp_amt_col]: opening_balance_row[col] = 0 # OB has 0 withdrawal/deposit
                 # For original balance column, keep its value (or NaN if it was missing/unparsable)
                 elif col == bal_col:
                     if bal_col not in opening_balance_row.columns: # If balance col itself was missing in OB row
                         opening_balance_row[col] = opening_balance_numeric if pd.notna(opening_balance_numeric) else np.nan
                     # else: it already exists, keep its value (parsed or NaN)
                 else: opening_balance_row[col] = np.nan # Default for others like text fields if not present


         # Ensure OB row has correct types matching df_processed where possible, BEFORE concatenation
         # Dates (convert existing date cols in OB row to datetime)
         for col in [date_col, val_dt_col]:
            if col in opening_balance_row.columns:
                opening_balance_row[col] = pd.to_datetime(opening_balance_row[col], errors='coerce')

         # Amounts (ensure numeric, apply same cleaning logic as before)
         cols_to_num = [wd_amt_col, dp_amt_col, bal_col] + ([calculated_balance_col] if calculated_balance_col in opening_balance_row.columns else [])
         for col in cols_to_num:
             if col in opening_balance_row.columns:
                 try:
                     # Apply same cleaning as main data for consistency before numeric conversion
                     ob_val_str = str(opening_balance_row[col].iloc[0]).strip().replace(',', '')
                     ob_val_str = re.sub(r'\s*(Cr|Dr)\.?$', '', ob_val_str)
                     opening_balance_row[col] = pd.to_numeric(ob_val_str, errors='coerce')
                 except:
                     opening_balance_row[col] = np.nan # Fallback if conversion fails

         # Text columns (ensure string, apply placeholder logic if needed)
         for col in text_cols_to_process: # [narr_col, ref_col]
            if col in opening_balance_row.columns:
                try:
                    text_val = str(opening_balance_row[col].iloc[0]).strip()
                    null_like_vals = ['nan', 'none', 'null', 'na', 'n/a', '#n/a', '<na>', 'nat', '-', '']
                    # If text is null-like or NaN after stripping, set to a specific OB placeholder
                    if pd.isna(text_val) or text_val.lower() in null_like_vals:
                         fill_value = 'Opening Balance Narration' if col == narr_col else ('OB Ref' if col == ref_col else np.nan)
                         opening_balance_row[col] = fill_value
                    else:
                        opening_balance_row[col] = text_val # Keep original OB text if valid
                except:
                    opening_balance_row[col] = 'OB Text Error' # Fallback

         # --- Concatenation ---
         try:
             # If transaction df exists, align OB columns to it
             if not df_processed.empty:
                 cols_order = df_processed.columns
                 # Reindex OB row to match df_processed exactly, adding missing columns as NaN
                 opening_balance_row_aligned = opening_balance_row.reindex(columns=cols_order)
                 # Concatenate: OB row first, then transactions
                 df_processed = pd.concat([opening_balance_row_aligned, df_processed], ignore_index=True)
                 print("     Successfully concatenated Opening Balance row to the top.")
             else:
                 # If only OB row exists, make it the main df
                 # Ensure it has at least the final required columns before proceeding
                 essential_cols = list(FINAL_BANK_COLS)
                 for col in essential_cols:
                     if col not in opening_balance_row.columns:
                         opening_balance_row[col] = np.nan # Add missing essential columns as NaN
                 # Use the OB row as the processed df
                 df_processed = opening_balance_row
                 print("     Using Opening Balance row as the main DataFrame (no transaction rows found).")

         except Exception as e:
             print(f"     WARN: Could not reliably concat opening balance row: {e}. OB Row might be lost or misaligned.")
             print(traceback.format_exc())
             # Fallback: If concatenation failed, try to proceed without OB row if transactions exist
             if opening_balance_row is not None and not df_processed.empty:
                 print("     Proceeding with transaction data only.")
             elif opening_balance_row is not None: # Only OB row existed and concat failed
                 print("     Cannot proceed as concatenation failed and no other data exists.")
                 return None # Critical error
    else:
        print("     No Opening Balance row was separated, nothing to concatenate.")


    # --- Step 10: Final Sort ---
    print("  10. Final sorting of data (including OB row if present)...")
    if date_col in df_processed.columns and pd.api.types.is_datetime64_any_dtype(df_processed[date_col]):
        # Sort by Date (NaNs/OB first), potentially add secondary sort keys if needed
        sort_keys = [date_col]
        # If Value Dt is reliable and present, use it as secondary sort
        # if val_dt_col in df_processed.columns and pd.api.types.is_datetime64_any_dtype(df_processed[val_dt_col]):
        #    sort_keys.append(val_dt_col)
        print(f"      Sorting by {sort_keys} (NaNs/OB first using mergesort)...")
        df_processed.sort_values(by=sort_keys, inplace=True, na_position='first', kind='mergesort')
        print(f"      Sorted by '{date_col}' (NaNs/OB first).")
    else:
        # Check if OB row is the only row and date is missing
        if len(df_processed) == 1 and opening_balance_row is not None and pd.isna(df_processed[date_col].iloc[0]):
             print(f"      Skipping sort: Only Opening Balance row exists with no valid date.")
        elif date_col not in df_processed.columns:
            print(f"      Cannot sort: Date column '{date_col}' is missing.")
        else:
             print(f"      Cannot sort: Date column '{date_col}' is not in datetime format after processing.")


    # --- Step 11: Select Final Columns ---
    print(f"  11. Selecting final columns: {FINAL_BANK_COLS}")
    # Check which of the desired final columns actually exist in the df *after* processing
    cols_to_keep = [col for col in FINAL_BANK_COLS if col in df_processed.columns]
    cols_missing_from_final = [col for col in FINAL_BANK_COLS if col not in df_processed.columns]

    if cols_missing_from_final:
        print(f"    WARN: The following desired final columns were not found in the processed data and will be omitted: {cols_missing_from_final}")

    if not cols_to_keep:
        print(f"    ERROR: None of the desired final columns {FINAL_BANK_COLS} were found in the processed DataFrame. Cannot create final output.")
        return None

    # Create the final DataFrame with only the selected columns
    df_final = df_processed[cols_to_keep].copy()
    print(f"    Selected {len(df_final.columns)} columns for final output: {list(df_final.columns)}")


    # --- Step 12: Final Inspection ---
    print("  12. Final Inspection...")
    print("      ------------------------------")
    print("      Final Bank Statement DataFrame Info (Selected Columns):")
    buffer = StringIO()
    df_final.info(memory_usage='deep', buf=buffer)
    print(buffer.getvalue())
    print("\n      Missing values after processing (Selected Columns):")
    missing_summary = df_final.isnull().sum()
    missing_summary = missing_summary[missing_summary > 0]
    if not missing_summary.empty:
        print(missing_summary.to_string())
    else:
        print("      No missing values remaining in final selected columns.")
    print(f"\n      Final Rows: {len(df_final)}")
    print("      ------------------------------")
    print(f"--- Finished Bank Statement Preprocessing: {filename} ---")

    return df_final


# =============================================================================
# 4. MAIN PROCESSING LOOP (BANK ONLY)
# =============================================================================
print("\n--- 4. Starting Bank Statement File Processing Loop ---")

processed_bank_count = 0
error_bank_count = 0

if not os.path.exists(bank_input_dir):
    print(f"ERROR: Bank input directory not found: {os.path.abspath(bank_input_dir)}. Please check the path and ensure Drive is mounted if using Colab.")
else:
    print(f"Reading Bank Statement files from: {os.path.abspath(bank_input_dir)}")
    try:
        all_items_in_dir = os.listdir(bank_input_dir)
        print(f"    Items found by os.listdir: {len(all_items_in_dir)}")
        # Filter more robustly for CSV
        all_bank_files = [item for item in all_items_in_dir if item.lower().endswith('.csv') and not item.startswith('.')]
        print(f"     Found {len(all_bank_files)} CSV files to process.")

        if not all_bank_files:
             print("     WARN: No CSV files found in the Bank input directory.")

        for filename in all_bank_files:
            input_path = os.path.join(bank_input_dir, filename)
            # Construct output filename robustly
            base_name, _ = os.path.splitext(filename)
            # Using a distinct suffix for this version
            output_filename = f"{base_name}_preprocessed_final_cols.csv"
            output_path = os.path.join(bank_output_dir, output_filename)

            try:
                print(f"\n>>> Reading Bank Statement file: {filename}")
                # Try reading with common encodings
                df_raw = None
                try: df_raw = pd.read_csv(input_path, encoding='utf-8')
                except UnicodeDecodeError:
                    print("      UTF-8 decoding failed, trying latin-1...")
                    try: df_raw = pd.read_csv(input_path, encoding='latin1')
                    except UnicodeDecodeError:
                        print("      latin-1 decoding failed, trying cp1252...")
                        try: df_raw = pd.read_csv(input_path, encoding='cp1252')
                        except UnicodeDecodeError as ude:
                             print(f"  ERROR: Could not decode file {filename} with utf-8, latin1, or cp1252. Skipping. Error: {ude}"); error_bank_count += 1
                             continue # Skip to next file
                except pd.errors.EmptyDataError:
                    print(f"  WARN: File {filename} is empty. Skipping."); error_bank_count += 1
                    continue # Skip to next file
                except FileNotFoundError:
                    print(f"  ERROR: File not found at {input_path}. Skipping."); error_bank_count += 1
                    continue # Skip to next file
                except Exception as read_err: # Catch other potential read errors
                    print(f"  ERROR: Failed to read CSV {filename}. Error: {read_err}")
                    print(traceback.format_exc()); error_bank_count += 1
                    continue

                if df_raw is None:
                    print(f"  ERROR: DataFrame df_raw is None after read attempts for {filename}. Skipping."); error_bank_count += 1
                    continue

                # --- Call Bank Preprocessing Function ---
                df_clean = preprocess_bank_statement_final_cols(df_raw, filename)

                if df_clean is not None and not df_clean.empty:
                    print(f"  --> Saving preprocessed Bank Statement file to: {output_path}")
                    df_clean.to_csv(output_path, index=False, encoding='utf-8')
                    processed_bank_count += 1
                elif df_clean is None:
                    print(f"  --> Skipping save for {filename} due to critical processing error (check logs above).")
                    error_bank_count += 1
                else: # df_clean is empty
                    print(f"  --> Skipping save for {filename} because the resulting DataFrame is empty after cleaning/selection.")
                    error_bank_count += 1

            except Exception as e: # Catch errors during processing or saving
                print(f"  ERROR: An unexpected error occurred while processing Bank file {filename}: {e}")
                print(traceback.format_exc()); error_bank_count += 1

    except FileNotFoundError:
         print(f"ERROR: Bank input directory itself not found during listdir: {bank_input_dir}")
         error_bank_count = -1 # Indicate directory level error
    except Exception as list_err:
        print(f"ERROR: Could not list or process Bank directory contents for {bank_input_dir}: {list_err}"); error_bank_count = -1

print(f"\nBank Statement Processing Summary: {processed_bank_count} files succeeded, {error_bank_count} files failed/skipped.")


# =============================================================================
# 5. FINAL SCRIPT COMPLETION MESSAGE (BANK)
# =============================================================================
print("\n" + "="*30 + " Bank Script Finished " + "="*30)
print("--- All Bank Statement preprocessing tasks complete! ---")
print(f"   Preprocessed Bank Statement files (final columns) saved in: {os.path.abspath(bank_output_dir)}")
print("="*70)

--- 1. Loading Libraries ---
--- 2. Setting Bank Statement Configuration ---
     Creating Bank output directory (if needed): /content/drive/MyDrive/Capstone Preprocessing/B2C/bank_preprocessed1
     Bank Input Directory: /content/drive/MyDrive/Capstone Preprocessing/B2C/bank_statements1
     Bank Output Directory: /content/drive/MyDrive/Capstone Preprocessing/B2C/bank_preprocessed1
     Required Bank Columns: ['Date', 'Narration', 'Chq./Ref.No.', 'Value Dt', 'Withdrawal Amt.', 'Deposit Amt.', 'Closing Balance']
     Desired Final Bank Columns: ['Date', 'Narration', 'Chq./Ref.No.', 'Value Dt', 'Withdrawal Amt.', 'Deposit Amt.', 'Closing Balance']

--- 3. Defining Bank Statement Preprocessing Function ---

--- 4. Starting Bank Statement File Processing Loop ---
Reading Bank Statement files from: /content/drive/MyDrive/Capstone Preprocessing/B2C/bank_statements1
    Items found by os.listdir: 100
     Found 100 CSV files to process.

>>> Reading Bank Statement file: 0L725_Bank_Statement.

<ipython-input-7-4d895411cc01>:160: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_processed[col].replace(['-', 'nil', 'null', 'N/A', 'NA', 'NaN', 'NaT', ''], np.nan, inplace=True)
<ipython-input-7-4d895411cc01>:160: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when

       NaNs before numeric conversion: 35606, After conversion: 35606
     Processing 'Deposit Amt.' column (converting from string)...
       NaNs before numeric conversion: 573, After conversion: 573
     Processing 'Closing Balance' column (converting from string)...
       NaNs before numeric conversion: 0, After conversion: 0
     Cleaning 'Narration' column...
       No common placeholder values found to replace in 'Narration'.
     Cleaning 'Chq./Ref.No.' column...
       No common placeholder values found to replace in 'Chq./Ref.No.'.
  4. Checking uniqueness of 'Chq./Ref.No.'...
    INFO: All non-missing values in 'Chq./Ref.No.' appear unique after cleaning.
  5. Handling remaining missing values...


<ipython-input-7-4d895411cc01>:209: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_processed[col].replace(['', '-', 'nil', 'null', 'N/A', 'NA', 'NaN', 'NaT'], np.nan, inplace=True)
<ipython-input-7-4d895411cc01>:282: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when

     Filling 35606 missing 'Withdrawal Amt.' values with 0.
     Filling 573 missing 'Deposit Amt.' values with 0.
  6. Checking for duplicate transaction rows...
     No duplicate transaction rows found based on key columns: ['Date', 'Narration', 'Withdrawal Amt.', 'Deposit Amt.', 'Chq./Ref.No.'].
  7. Adding temporary features (date components, transaction type)...
     Added temporary date features.
     Added temporary transaction type.
  8. Attempting to recalculate closing balance (into temporary column)...
     Using parsed Opening Balance: 62607.39
       Sorting by ['Date'] for calculation...
     Successfully added temporary 'calculated_balance' column.
     Comparison with original 'Closing Balance' (36179 comparable rows): Max diff: 0.00, Mean diff: 0.00
       Calculated balance matches original balance (within 0.01 tolerance) where comparable.
  9. Handling Opening Balance row concatenation...
     Attempting to reintegrate Opening Balance row...
     Successfully concate

...


>>> Reading Bank Statement file: ZKLOO_Bank_Statement.csv

--- Starting Bank Statement Preprocessing for: ZKLOO_Bank_Statement.csv ---
     Initial raw columns: ['Date', 'Narration', 'Chq./Ref.No.', 'Value Dt', 'Withdrawal Amt.', 'Deposit Amt.', 'Closing Balance']
  1. Verifying required columns...
     Found required Bank columns: ['Date', 'Narration', 'Chq./Ref.No.', 'Value Dt', 'Withdrawal Amt.', 'Deposit Amt.', 'Closing Balance']
  2. Checking for and separating Opening Balance row...
     Detected and separated potential Opening Balance row based on Narration ('opening balance') and missing Date or zero/missing Amounts.
  3. Processing Data Types & Cleaning...
     Processing 'Date' column...
       Invalid dates after DD-MM-YYYY format: 0
       NaNs/NaTs before parsing: 0, After parsing: 0
     Processing 'Value Dt' column...
       Invalid dates after DD-MM-YYYY format: 0
       NaNs/NaTs before parsing: 0, After parsing: 0
     Processing 'Withdrawal Amt.' column (converting 

<ipython-input-7-4d895411cc01>:160: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_processed[col].replace(['-', 'nil', 'null', 'N/A', 'NA', 'NaN', 'NaT', ''], np.nan, inplace=True)
<ipython-input-7-4d895411cc01>:160: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when

     Successfully concatenated Opening Balance row to the top.
  10. Final sorting of data (including OB row if present)...
      Sorting by ['Date'] (NaNs/OB first using mergesort)...
      Sorted by 'Date' (NaNs/OB first).
  11. Selecting final columns: ['Date', 'Narration', 'Chq./Ref.No.', 'Value Dt', 'Withdrawal Amt.', 'Deposit Amt.', 'Closing Balance']
    Selected 7 columns for final output: ['Date', 'Narration', 'Chq./Ref.No.', 'Value Dt', 'Withdrawal Amt.', 'Deposit Amt.', 'Closing Balance']
  12. Final Inspection...
      ------------------------------
      Final Bank Statement DataFrame Info (Selected Columns):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13019 entries, 0 to 13018
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             13019 non-null  datetime64[ns]
 1   Narration        13019 non-null  object        
 2   Chq./Ref.No.     13019 non-null  object      

<ipython-input-7-4d895411cc01>:160: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_processed[col].replace(['-', 'nil', 'null', 'N/A', 'NA', 'NaN', 'NaT', ''], np.nan, inplace=True)
<ipython-input-7-4d895411cc01>:160: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when

       Invalid dates after DD-MM-YYYY format: 0
       NaNs/NaTs before parsing: 0, After parsing: 0
     Processing 'Withdrawal Amt.' column (converting from string)...
       NaNs before numeric conversion: 12869, After conversion: 12869
     Processing 'Deposit Amt.' column (converting from string)...
       NaNs before numeric conversion: 782, After conversion: 782
     Processing 'Closing Balance' column (converting from string)...
       NaNs before numeric conversion: 0, After conversion: 0
     Cleaning 'Narration' column...
       No common placeholder values found to replace in 'Narration'.
     Cleaning 'Chq./Ref.No.' column...
       No common placeholder values found to replace in 'Chq./Ref.No.'.
  4. Checking uniqueness of 'Chq./Ref.No.'...
    INFO: All non-missing values in 'Chq./Ref.No.' appear unique after cleaning.
  5. Handling remaining missing values...
     Filling 12869 missing 'Withdrawal Amt.' values with 0.
     Filling 782 missing 'Deposit Amt.' values with 

In [ ]:
import pandas as pd
import sys
import csv

input_filename = '/content/drive/MyDrive/Capstone Preprocessing/B2C/company_data_with_official_gst_igst.csv' # Your ORIGINAL file
# Using a new name to indicate formatting attempt
output_filename = '/content/drive/MyDrive/Capstone Preprocessing/B2C/processed_company_data_formatted.csv'

# List of columns to process
rate_columns = ['CGST (%)', 'SGST (%)', 'IGST (%)']

# --- 1. Load Data ---
try:
    df = pd.read_csv(input_filename)
    print(f"Loaded data from '{input_filename}'.")
except FileNotFoundError:
    print(f"Error: Input file not found at '{input_filename}'")
    sys.exit(1)
except Exception as e:
    print(f"Error reading file '{input_filename}': {e}")
    sys.exit(1)

# --- 2. Preprocessing Function (with Typo Correction) ---
def clean_correct_deduplicate_rates(rate_cell_value):
    """
    Cleans, corrects specific typos (2009->9, 2006->6),
    and removes duplicate rates within a comma-separated string cell.
    """
    try:
        rate_str = str(rate_cell_value).strip()
    except Exception:
        return rate_cell_value # Return original if conversion fails

    if not rate_str:
        return "" # Return empty string for consistency

    parts = [part.strip() for part in rate_str.split(',') if part.strip()]
    if not parts:
        return ""

    # Correction Step
    corrected_parts = []
    for part in parts:
        if part == '2009':
            corrected_parts.append('9')
        elif part == '2006':
            corrected_parts.append('6')
        else:
            corrected_parts.append(part)

    # Deduplication Step
    unique_parts = []
    for part in corrected_parts:
        if part not in unique_parts:
            unique_parts.append(part)

    return ', '.join(unique_parts)

# --- 3. Apply Function to All Rows ---
print("Processing rate columns (correcting typos and removing duplicates)...")
processed_count = 0
for col in rate_columns:
    if col in df.columns:
        # Ensure column is treated as string, handle NaN, apply cleaning function
        df[col] = df[col].fillna('').astype(str).apply(clean_correct_deduplicate_rates)
        processed_count += 1
    else:
        print(f"Warning: Column '{col}' not found.")

if processed_count == 0:
     print(f"Error: None of the columns {rate_columns} were found. Exiting.")
     sys.exit(1)

print("Processing complete.")

# --- 4. Save Entire Processed Data with Quoting ---
try:
    # Save the DataFrame, quoting non-numeric fields to encourage text treatment
    df.to_csv(
        output_filename,
        index=False,
        encoding='utf-8',
        quoting=csv.QUOTE_NONNUMERIC # <<< Key change for formatting
        )
    print(f"Processed data ({len(df)} rows) saved to '{output_filename}'.")
    print("NOTE: Alignment depends on the viewing software (e.g., Excel).")
    print("      Using QUOTE_NONNUMERIC encourages text alignment for processed columns.")

except Exception as e:
    print(f"Error saving file '{output_filename}': {e}")

print("Script finished.")

Loaded data from '/content/drive/MyDrive/Capstone Preprocessing/B2C/company_data_with_official_gst_igst.csv'.
Processing rate columns (correcting typos and removing duplicates)...
Processing complete.
Processed data (100 rows) saved to '/content/drive/MyDrive/Capstone Preprocessing/B2C/processed_company_data_formatted.csv'.
NOTE: Alignment depends on the viewing software (e.g., Excel).
      Using QUOTE_NONNUMERIC encourages text alignment for processed columns.
Script finished.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import glob
import os

# Path to your raw folder
folder_path = "/content/drive/MyDrive/Capstone Preprocessing/B2C/b2c_sales_fraud"

# Create new folder at the same level as b2c_sales_fraud
parent_folder = os.path.dirname(folder_path)  # this will be .../B2C
cleaned_folder = os.path.join(parent_folder, "b2c_sales_fraud_cleaned")
os.makedirs(cleaned_folder, exist_ok=True)

# Function to clean 'Applicable Rate (%)'
def clean_rate(x):
    if pd.isna(x):  # handle NaN
        return None
    if isinstance(x, str) and "+" in x:
        try:
            return sum(map(float, x.split("+")))  # allow decimals like 2.5+9
        except:
            return None
    try:
        return float(x)
    except:
        return None

# Loop through all CSV files in raw folder
for file in glob.glob(os.path.join(folder_path, "*.csv")):
    print(f"Processing {file} ...")

    df = pd.read_csv(file)

    # Clean Applicable Rate (%)
    df["Applicable Rate (%)"] = df["Applicable Rate (%)"].apply(clean_rate)

    # Fill missing GSTIN with "Nil"
    df["E-Commerce GSTIN"] = df["E-Commerce GSTIN"].fillna("Nil")

    # Convert numeric columns
    df["Rate"] = pd.to_numeric(df["Rate"], errors="coerce")
    df["Taxable Value"] = pd.to_numeric(df["Taxable Value"], errors="coerce")
    df["Cess Amount"] = pd.to_numeric(df["Cess Amount"], errors="coerce")

    # Save cleaned file into new folder
    base_name = os.path.basename(file)
    name, ext = os.path.splitext(base_name)
    cleaned_file = os.path.join(cleaned_folder, f"{name}_cleaned{ext}")

    df.to_csv(cleaned_file, index=False)
    print(f"✅ Saved cleaned file: {cleaned_file}")

print("🎉 All files processed. Cleaned versions are in:", cleaned_folder)


Processing /content/drive/MyDrive/Capstone Preprocessing/B2C/b2c_sales_fraud/30T9N_B2C_Sales.csv ...
✅ Saved cleaned file: /content/drive/MyDrive/Capstone Preprocessing/B2C/b2c_sales_fraud_cleaned/30T9N_B2C_Sales_cleaned.csv
Processing /content/drive/MyDrive/Capstone Preprocessing/B2C/b2c_sales_fraud/31YIH_B2C_Sales.csv ...
✅ Saved cleaned file: /content/drive/MyDrive/Capstone Preprocessing/B2C/b2c_sales_fraud_cleaned/31YIH_B2C_Sales_cleaned.csv
Processing /content/drive/MyDrive/Capstone Preprocessing/B2C/b2c_sales_fraud/3J2D5_B2C_Sales.csv ...
✅ Saved cleaned file: /content/drive/MyDrive/Capstone Preprocessing/B2C/b2c_sales_fraud_cleaned/3J2D5_B2C_Sales_cleaned.csv
Processing /content/drive/MyDrive/Capstone Preprocessing/B2C/b2c_sales_fraud/0Y9DO_B2C_Sales.csv ...
✅ Saved cleaned file: /content/drive/MyDrive/Capstone Preprocessing/B2C/b2c_sales_fraud_cleaned/0Y9DO_B2C_Sales_cleaned.csv
Processing /content/drive/MyDrive/Capstone Preprocessing/B2C/b2c_sales_fraud/60RJI_B2C_Sales.csv ...